# Text generation with an RNN

### Import TensorFlow and other libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import os
import time

### Download the dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_to_file = '/content/drive/MyDrive/Colab Notebooks/KindlePreprocessed.txt'

### Read the data


In [ ]:
# read and decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
text = text.replace('\n', ' ')
print(f'Length of text: {len(text)} characters')

Length of text: 10172244 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

silly predictable romance, read it in an afternoon with a hangover, that's all I'd recommend this author for. Fortunately it was free and I test drove my new Kindle. This is not a hockey romance! It is Completely misleading and not a great story. I a


In [ ]:
# unique characters 
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

96 unique characters


## Process the text

In [ ]:
# convert strings to numerical representation 
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
# invert representation to recover readable strings
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
# join chars into strings
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [1]:
# function that takes a sequence, duplicates and shifts it to align the input and label
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [3]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
# create training batches w shuffled data to. feed into model
BATCH_SIZE = 64

# buffer size to shuffle the dataset
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

In [ ]:
# length of the vocabulary in chars
vocab_size = len(vocab)
# embedding dimension
embedding_dim = 256
# number of RNN units
rnn_units = 1024

In [ ]:
# build the model (keras.Model subclass)
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  24832     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [ ]:
# attach an optimizer and loss function
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 97)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.575968


In [ ]:
# newly initialized model should have an exponential mean loss approx equal to vocab size
tf.exp(mean_loss).numpy()

97.12198

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# configure checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
# train the model
EPOCHS = 10

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
1573/1573 [==============================] - 219s 137ms/step - loss: 1.5956
Epoch 2/10
1573/1573 [==============================] - 217s 137ms/step - loss: 1.2173
Epoch 3/10
1573/1573 [==============================] - 218s 138ms/step - loss: 1.1575
Epoch 4/10
1573/1573 [==============================] - 217s 137ms/step - loss: 1.1247
Epoch 5/10
1573/1573 [==============================] - 218s 138ms/step - loss: 1.1023
Epoch 6/10
1573/1573 [==============================] - 217s 137ms/step - loss: 1.0864
Epoch 7/10
1573/1573 [==============================] - 218s 138ms/step - loss: 1.0748
Epoch 8/10
1573/1573 [==============================] - 217s 137ms/step - loss: 1.0668
Epoch 9/10
1573/1573 [==============================] - 218s 138ms/step - loss: 1.0612
Epoch 10/10
1573/1573 [==============================] - 218s 138ms/step - loss: 1.0582


## Generate text

In [ ]:
# single step prediction
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()


    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
# generate 10 product reviews using the trained model
start = time.time()
states = None
next_char = tf.constant(['The book', 'The book', 'The book', 'The book', 'The book', 'The book', 'The book', 'The book', 'The book', 'The book'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
words = tf.strings.join(result).numpy().decode("utf-8")
print(result, ' ' + '_'*80)
print('\nRun time:', end - start)
print(words)
text_file = open("/content/drive/MyDrive/Colab Notebooks/results.txt", "w")
n = text_file.write(words)
text_file.close()

tf.Tensor(
[b"The book is subtised. Until I just made Edjortice young pleasure against what the hell. The majority only be so far of Barrow's the same protection eventually seem to want to comment where they are.  Lilliana Timgi wakes compassion to help you hooked revenged on another to torn motthe Russ! And I zouth competerly escape, to the point or the storyline. This is a wonderful story of love that Vic Felton has been with him, he's supposed to be the verge of town of ones jule and sets out to just en if she. Looking forward to reading more books in the series! I will tell about Geri's Justice Hut, forth; this one also thinks so that could take a colded weekend more clever.  She meets the romantic aftermath for a first love and turn down everything finally feeling for Abra.  Would have told her to stop Rexen in for an honesty one.  The two punchs make me a whole lot of twists and turns that are looking for being bride. I really enjoyed this book. I was not disappointed in the next